In [ ]:
import warnings
warnings.filterwarnings("ignore")

import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import StackingRegressor
from sklearn.linear_model import RidgeCV
from sklearn.metrics import r2_score, root_mean_squared_error
import optuna

# GPU-compatible libraries
import xgboost as xgb
import lightgbm as lgb
from catboost import CatBoostRegressor

# ======================
# 1) Dati + preprocessing
# ======================
from sklearn.datasets import fetch_california_housing
raw = fetch_california_housing()
data = pd.DataFrame(raw.data, columns=raw.feature_names)
data["MedHouseVal"] = raw.target

# Feature engineering
data["Ave_Room_Bed"] = data["AveRooms"] / (data["AveBedrms"] + 1e-6)
data["RoomsPerPerson"] = data["AveRooms"] / (data["Population"] + 1.0)
data["Lat_Long"] = data["Latitude"] * data["Longitude"]

X = data.drop(columns=["MedHouseVal"])
y = data["MedHouseVal"].values

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# ======================
# 2) Funzioni obiettivo Optuna (GPU)
# ======================
def objective_xgb(trial):
    param = {
        "n_estimators": trial.suggest_int("n_estimators", 100, 500),
        "max_depth": trial.suggest_int("max_depth", 3, 10),
        "learning_rate": trial.suggest_float("learning_rate", 0.01, 0.2, log=True),
        "subsample": trial.suggest_float("subsample", 0.6, 1.0),
        "colsample_bytree": trial.suggest_float("colsample_bytree", 0.6, 1.0),
        "reg_alpha": trial.suggest_float("reg_alpha", 1e-6, 10.0, log=True),
        "reg_lambda": trial.suggest_float("reg_lambda", 1e-6, 10.0, log=True),
        "random_state": 42,
        "tree_method": "gpu_hist",      # GPU
        "predictor": "gpu_predictor",   # GPU
        "n_jobs": -1
    }
    model = xgb.XGBRegressor(**param)
    scores = cross_val_score(model, X_train_scaled, y_train, cv=3,
                             scoring="neg_root_mean_squared_error", n_jobs=1)
    return -scores.mean()

def objective_lgb(trial):
    param = {
        "n_estimators": trial.suggest_int("n_estimators", 100, 1000),
        "num_leaves": trial.suggest_int("num_leaves", 16, 256),
        "max_depth": trial.suggest_int("max_depth", 3, 16),
        "learning_rate": trial.suggest_float("learning_rate", 0.005, 0.15, log=True),
        "subsample": trial.suggest_float("subsample", 0.6, 1.0),
        "colsample_bytree": trial.suggest_float("colsample_bytree", 0.6, 1.0),
        "reg_alpha": trial.suggest_float("reg_alpha", 1e-6, 10.0, log=True),
        "reg_lambda": trial.suggest_float("reg_lambda", 1e-6, 10.0, log=True),
        "random_state": 42,
        "device": "gpu",   # GPU
        "n_jobs": -1
    }
    model = lgb.LGBMRegressor(**param)
    scores = cross_val_score(model, X_train_scaled, y_train, cv=3,
                             scoring="neg_root_mean_squared_error", n_jobs=1)
    return -scores.mean()

def objective_cat(trial):
    param = {
        "iterations": trial.suggest_int("iterations", 500, 1500),
        "learning_rate": trial.suggest_float("learning_rate", 0.01, 0.1),
        "depth": trial.suggest_int("depth", 4, 10),
        "l2_leaf_reg": trial.suggest_float("l2_leaf_reg", 1, 10),
        "random_state": 42,
        "task_type": "GPU",   # GPU
        "devices": "0:1",     # GPU device(s)
        "verbose": 0
    }
    model = CatBoostRegressor(**param)
    scores = cross_val_score(model, X_train_scaled, y_train, cv=3,
                             scoring="neg_root_mean_squared_error", n_jobs=1)
    return -scores.mean()

# ======================
# 3) Esegui ottimizzazione Optuna
# ======================
study_xgb = optuna.create_study(direction="minimize")
study_xgb.optimize(objective_xgb, n_trials=20, show_progress_bar=False)

study_lgb = optuna.create_study(direction="minimize")
study_lgb.optimize(objective_lgb, n_trials=20, show_progress_bar=False)

study_cat = optuna.create_study(direction="minimize")
study_cat.optimize(objective_cat, n_trials=15, show_progress_bar=False)

best_xgb_params = study_xgb.best_params
best_lgb_params = study_lgb.best_params
best_cat_params = study_cat.best_params

print("Best XGB:", best_xgb_params)
print("Best LGB:", best_lgb_params)
print("Best Cat:", best_cat_params)

# ======================
# 4) Costruisci modelli finali
# ======================
xgb_est = xgb.XGBRegressor(**best_xgb_params, random_state=42, tree_method="gpu_hist", predictor="gpu_predictor")
lgb_est = lgb.LGBMRegressor(**best_lgb_params, random_state=42, device="gpu")
cat_est = CatBoostRegressor(**best_cat_params, task_type="GPU", devices="0:1", verbose=0)

xgb_est.fit(X_train_scaled, y_train)
lgb_est.fit(X_train_scaled, y_train)
cat_est.fit(X_train_scaled, y_train)

# ======================
# 5) Stacking
# ======================
estimators = [("xgb", xgb_est), ("lgb", lgb_est), ("cat", cat_est)]
stack = StackingRegressor(
    estimators=estimators,
    final_estimator=RidgeCV(),
    cv=5,
    n_jobs=-1
)

stack.fit(X_train_scaled, y_train)
pred_stack = stack.predict(X_test_scaled)
rmse_stack = root_mean_squared_error(y_test, pred_stack)
r2_stack = r2_score(y_test, pred_stack)

print(f"STACK RMSE: {rmse_stack:.4f}, R2: {r2_stack:.4f}")


[I 2025-12-09 17:21:25,495] A new study created in memory with name: no-name-6b5a5910-3b7c-4cf8-a542-e7a322688344
[I 2025-12-09 17:21:27,916] Trial 0 finished with value: 0.49544998943977525 and parameters: {'n_estimators': 298, 'max_depth': 5, 'learning_rate': 0.023017410539018155, 'subsample': 0.8837345337915046, 'colsample_bytree': 0.8253222242186982, 'reg_alpha': 1.5004425456864625e-06, 'reg_lambda': 8.342951058259236e-05}. Best is trial 0 with value: 0.49544998943977525.
[I 2025-12-09 17:21:32,479] Trial 1 finished with value: 0.48221668798081163 and parameters: {'n_estimators': 443, 'max_depth': 6, 'learning_rate': 0.014470181162732272, 'subsample': 0.6536910220959432, 'colsample_bytree': 0.8934231330195344, 'reg_alpha': 0.002938285614431645, 'reg_lambda': 4.578121933675406e-06}. Best is trial 1 with value: 0.48221668798081163.
[I 2025-12-09 17:21:44,610] Trial 2 finished with value: 0.4670621527614512 and parameters: {'n_estimators': 283, 'max_depth': 10, 'learning_rate': 0.0252

[LightGBM] [Info] This is the GPU trainer!!
[LightGBM] [Info] Total Bins 2602
[LightGBM] [Info] Number of data points in the train set: 11008, number of used features: 11
[LightGBM] [Info] Using GPU Device: NVIDIA GeForce RTX 4060 Ti, Vendor: NVIDIA Corporation
[LightGBM] [Info] Compiling OpenCL Kernel with 256 bins...
[LightGBM] [Info] GPU programs have been built
[LightGBM] [Info] Size of histogram bin entry: 8
[LightGBM] [Info] 11 dense feature groups (0.13 MB) transferred to GPU in 0.000902 secs. 0 sparse feature groups
[LightGBM] [Info] Start training from score 2.064393
[LightGBM] [Info] This is the GPU trainer!!
[LightGBM] [Info] Total Bins 2602
[LightGBM] [Info] Number of data points in the train set: 11008, number of used features: 11
[LightGBM] [Info] Using GPU Device: NVIDIA GeForce RTX 4060 Ti, Vendor: NVIDIA Corporation
[LightGBM] [Info] Compiling OpenCL Kernel with 256 bins...
[LightGBM] [Info] GPU programs have been built
[LightGBM] [Info] Size of histogram bin entry: 8


[I 2025-12-09 17:23:17,822] Trial 0 finished with value: 0.45896222015991595 and parameters: {'n_estimators': 420, 'num_leaves': 43, 'max_depth': 16, 'learning_rate': 0.1458620329805018, 'subsample': 0.9860192568465089, 'colsample_bytree': 0.9249903968389634, 'reg_alpha': 0.4948812706321945, 'reg_lambda': 0.04163744219389}. Best is trial 0 with value: 0.45896222015991595.


[LightGBM] [Info] This is the GPU trainer!!
[LightGBM] [Info] Total Bins 2602
[LightGBM] [Info] Number of data points in the train set: 11008, number of used features: 11
[LightGBM] [Info] Using GPU Device: NVIDIA GeForce RTX 4060 Ti, Vendor: NVIDIA Corporation
[LightGBM] [Info] Compiling OpenCL Kernel with 256 bins...
[LightGBM] [Info] GPU programs have been built
[LightGBM] [Info] Size of histogram bin entry: 8
[LightGBM] [Info] 11 dense feature groups (0.13 MB) transferred to GPU in 0.000852 secs. 0 sparse feature groups
[LightGBM] [Info] Start training from score 2.064393
[LightGBM] [Info] This is the GPU trainer!!
[LightGBM] [Info] Total Bins 2602
[LightGBM] [Info] Number of data points in the train set: 11008, number of used features: 11
[LightGBM] [Info] Using GPU Device: NVIDIA GeForce RTX 4060 Ti, Vendor: NVIDIA Corporation
[LightGBM] [Info] Compiling OpenCL Kernel with 256 bins...
[LightGBM] [Info] GPU programs have been built
[LightGBM] [Info] Size of histogram bin entry: 8


[I 2025-12-09 17:23:31,773] Trial 1 finished with value: 0.5156244821371937 and parameters: {'n_estimators': 714, 'num_leaves': 21, 'max_depth': 7, 'learning_rate': 0.005521751811186665, 'subsample': 0.694556410623593, 'colsample_bytree': 0.6261816266200942, 'reg_alpha': 0.09575273435280486, 'reg_lambda': 4.463656451382153}. Best is trial 0 with value: 0.45896222015991595.


...

[LightGBM] [Info] This is the GPU trainer!!
[LightGBM] [Info] Total Bins 2602
[LightGBM] [Info] Number of data points in the train set: 11008, number of used features: 11
[LightGBM] [Info] Using GPU Device: NVIDIA GeForce RTX 4060 Ti, Vendor: NVIDIA Corporation
[LightGBM] [Info] Compiling OpenCL Kernel with 256 bins...
[LightGBM] [Info] GPU programs have been built
[LightGBM] [Info] Size of histogram bin entry: 8
[LightGBM] [Info] 11 dense feature groups (0.13 MB) transferred to GPU in 0.001020 secs. 0 sparse feature groups
[LightGBM] [Info] Start training from score 2.064393
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits wit

[I 2025-12-09 17:39:06,687] Trial 19 finished with value: 0.44332805898105887 and parameters: {'n_estimators': 835, 'num_leaves': 172, 'max_depth': 12, 'learning_rate': 0.022939954463489023, 'subsample': 0.7602055671751626, 'colsample_bytree': 0.6696394664111355, 'reg_alpha': 0.07246715421257888, 'reg_lambda': 1.5988379921499036e-05}. Best is trial 19 with value: 0.44332805898105887.
[I 2025-12-09 17:39:06,688] A new study created in memory with name: no-name-876fd978-53a7-492a-ac34-b6503008d406
[I 2025-12-09 17:40:04,096] Trial 0 finished with value: 0.4532930393539933 and parameters: {'iterations': 1058, 'learning_rate': 0.09350716513068133, 'depth': 7, 'l2_leaf_reg': 4.925167659419896}. Best is trial 0 with value: 0.4532930393539933.
[I 2025-12-09 17:40:50,369] Trial 1 finished with value: 0.4605364654195936 and parameters: {'iterations': 1165, 'learning_rate': 0.081932141015925, 'depth': 5, 'l2_leaf_reg': 9.17064849895506}. Best is trial 0 with value: 0.4532930393539933.
[I 2025-12

Best XGB: {'n_estimators': 360, 'max_depth': 9, 'learning_rate': 0.03821528290773023, 'subsample': 0.683533094403276, 'colsample_bytree': 0.7187917338264955, 'reg_alpha': 3.1906582182306593, 'reg_lambda': 0.022554881280094124}
Best LGB: {'n_estimators': 835, 'num_leaves': 172, 'max_depth': 12, 'learning_rate': 0.022939954463489023, 'subsample': 0.7602055671751626, 'colsample_bytree': 0.6696394664111355, 'reg_alpha': 0.07246715421257888, 'reg_lambda': 1.5988379921499036e-05}
Best Cat: {'iterations': 1470, 'learning_rate': 0.06684189434508243, 'depth': 10, 'l2_leaf_reg': 6.978097064401743}
[LightGBM] [Info] This is the GPU trainer!!
[LightGBM] [Info] Total Bins 2603
[LightGBM] [Info] Number of data points in the train set: 16512, number of used features: 11
[LightGBM] [Info] Using GPU Device: NVIDIA GeForce RTX 4060 Ti, Vendor: NVIDIA Corporation
[LightGBM] [Info] Compiling OpenCL Kernel with 256 bins...
[LightGBM] [Info] GPU programs have been built
[LightGBM] [Info] Size of histogram b

Exception ignored on calling ctypes callback function: <function _log_callback at 0x000002777F66CCC0>
Traceback (most recent call last):
  File "c:\Python312\Lib\site-packages\lightgbm\basic.py", line 287, in _log_callback
    def _log_callback(msg: bytes) -> None:
    
KeyboardInterrupt: 


No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with po

**RISULTATO**

RMSE Baseline:    0.4718
RMSE Ottimizzato: 0.4300

param = {
        'verbosity': 0,
        'objective': 'reg:squarederror',
        'eval_metric': 'rmse', # Metrica per early stopping e pruning
        'tree_method': 'hist',
        
        # --- Struttura ---
        # California ha interazioni complesse, permettiamo alberi più profondi
        'max_depth': trial.suggest_int('max_depth', 2, 15),
        # Importante per evitare overfitting su outlier di prezzo
        'min_child_weight': trial.suggest_int('min_child_weight', 1, 20),
        'gamma': trial.suggest_float('gamma', 1e-8, 1.0, log=True),
        
        # --- Randomness ---
        'subsample': trial.suggest_float('subsample', 0.6, 1.0),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.6, 1.0),
        
        # --- Regolarizzazione ---
        # Fondamentale nella regressione per non inseguire i prezzi estremi
        'lambda': trial.suggest_float('lambda', 1e-9, 25.0, log=True),
        'alpha': trial.suggest_float('alpha', 1e-9, 25.0, log=True),
        
        # --- Learning ---
        'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.3, log=True),
    }

study.optimize(objective, n_trials=70, timeout=600)

=============================================================================== <br>
RMSE Baseline:    0.4718
RMSE Ottimizzato: 0.4310

con scaler

param = {
        'verbosity': 0,
        'objective': 'reg:squarederror',
        'eval_metric': 'rmse', # Metrica per early stopping e pruning
        'tree_method': 'hist',
        
        # --- Struttura ---
        # California ha interazioni complesse, permettiamo alberi più profondi
        'max_depth': trial.suggest_int('max_depth', 2, 20),
        # Importante per evitare overfitting su outlier di prezzo
        'min_child_weight': trial.suggest_int('min_child_weight', 1, 25),
        'gamma': trial.suggest_float('gamma', 1e-9, 1.5, log=True),
        
        # --- Randomness ---
        'subsample': trial.suggest_float('subsample', 0.6, 1.0),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.6, 1.0),
        
        # --- Regolarizzazione ---
        # Fondamentale nella regressione per non inseguire i prezzi estremi
        'lambda': trial.suggest_float('lambda', 1e-6, 20.0, log=True),
        'alpha': trial.suggest_float('alpha', 1e-6, 20.0, log=True),
        
        # --- Learning ---
        'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.8),
    }

study.optimize(objective, n_trials=200, timeout=600)

=============================================================================== <br>

RMSE Baseline:    0.3522
RMSE Ottimizzato: 0.3379

con scaler

study.optimize(objective, n_trials=200, timeout=600)

param = {
        'verbosity': 0,
        'objective': 'reg:squarederror',
        'eval_metric': 'rmse', # Metrica per early stopping e pruning
        'tree_method': 'hist',
        
        # --- Struttura ---
        # California ha interazioni complesse, permettiamo alberi più profondi
        'max_depth': trial.suggest_int('max_depth', 2, 20),
        # Importante per evitare overfitting su outlier di prezzo
        'min_child_weight': trial.suggest_int('min_child_weight', 1, 25),
        'gamma': trial.suggest_float('gamma', 1e-7, 1.5, log=True),
        
        # --- Randomness ---
        'subsample': trial.suggest_float('subsample', 0.6, 1.0),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.6, 1.0),
        
        # --- Regolarizzazione ---
        # Fondamentale nella regressione per non inseguire i prezzi estremi
        'lambda': trial.suggest_float('lambda', 1e-9, 15.0, log=True),
        'alpha': trial.suggest_float('alpha', 1e-9, 15.0, log=True),
        
        # --- Learning ---
        'learning_rate': trial.suggest_float('learning_rate', 0.001, 0.6),
    }

#rimozione outlier
numeric_cols = data.select_dtypes(include=["float", "int"]).columns.tolist()

for col in numeric_cols:
    Q1 = data[col].quantile(0.3)
    Q3 = data[col].quantile(0.7)
    IQR = Q3 - Q1
    lower, upper = Q1 - 1.5 * IQR, Q3 + 1.5 * IQR

    data[col] = np.where(data[col].between(lower, upper), data[col], np.nan)

data = data.dropna()

#features combinate
data['Ave_Room_Bed'] = data['AveRooms'] / data['AveBedrms']
data['HouseRooms'] = data['HouseAge'] / data['AveRooms']

Miglior RMSE trovato da Optuna (Validation): 0.3544
Migliori parametri: {'max_depth': 13, 'min_child_weight': 23, 'gamma': 3.8838420870962745e-06, 'subsample': 0.7570628531701007, 'colsample_bytree': 0.9109736816364161, 'lambda': 0.02307771384295654, 'alpha': 0.9454327224813569, 'learning_rate': 0.06885420667941514}